In [8]:
import math
from pyquil import Program, get_qc
from pyquil.gates import H
from six.moves import range

In [13]:
def qubits_needed(numbSides):
    """
    The number of qubits needed for faces of the dice
    """
    return int(math.ceil(math.log(numbSides, 2)))

In [15]:
def get_qvm(numbSides):
    """
    QVM to simulate the number of sides
    """
    return get_qc(f"{qubits_needed(numbSides)}q-qvm")

In [22]:
def die_program(numbSsides):
    """
    quantum program to roll a dice of n faces.
    """
    prog = Program()
    n_qubits = qubits_needed(numbSides)
    ro = prog.declare('ro', 'BIT', n_qubits)
    # Hadamard initialize.
    for q in range(n_qubits):
        prog.inst(H(q))
    # Measure everything.
    for q in range(n_qubits):
        prog.measure(q, ro[q])
    return prog

In [23]:
def process_results(results):
    """
    convert number of faces digit binary result from the QVM to a value on a dice
    """
    raw_results = results[0]
    processing_result = 0
    for each_qubit_measurement in raw_results:
        processing_result = 2*processing_result + each_qubit_measurement
    # Convert from 0 indexed to 1 indexed
    dieValue = processing_result + 1
    return dieValue

In [24]:
def roll_die(qvm, numbSides):
    """
    roll an numb-sided quantum dice
    """
    die_compiled = qvm.compile(die_program(numbSides))
    return process_results(qvm.run(die_compiled))

In [25]:
if __name__ == '__main__':
    numbSides = int(input("Please enter number of sides: "))
    qvm = get_qvm(numbSides)
    die_value = roll_die(qvm, numbSides)
    while die_value > numbSides: die_value = roll_die(qvm, numbSides)
    print(f"The result is: {die_value}")

Please enter number of sides: 2


TimeoutError: Timeout on client tcp://127.0.0.1:5555, method name quil_to_native_quil, class info: <rpcq._client.Client object at 0xa1fade2e8>